## Marketing Strategy Analysis
#### Programming Script and Technical Report

### Table of Content
1. Introduction
2. Data Loading and Quality
3. Feature Additions and Engineering
4. Exploratory Data Analysis and Statistical Analysis
5. Final Recommendations (Optimal Sales)

### 1. Introduction

- What is the impact of each marketing strategy and sales visit on Sales (Amount Collected)?
- Is the same strategy valid for all the different client types ?


### 2. Data Loading and Quality Checks

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats

In [2]:
from google.colab import files
uploaded = files.upload()

Saving bike_sales_data.csv to bike_sales_data.csv


In [3]:
import io
data = pd.read_csv(io.BytesIO(uploaded['bike_sales_data.csv']))
# Dataset is now stored in a Pandas Dataframe

In [4]:
#data=pd.read_csv('Campaign Data.csv')
data.columns

Index(['Week', 'sales', 'branded_search_spend', 'nonbranded_search_spend',
       'facebook_spend', 'print_spend', 'ooh_spend', 'tv_spend',
       'radio_spend'],
      dtype='object')

In [5]:
data.head()

,Week,sales,branded_search_spend,nonbranded_search_spend,facebook_spend,print_spend,ooh_spend,tv_spend,radio_spend
0,7/23/17,58850.0,1528.8,463.32,802.620,0,0,0,0
1,7/30/17,62050.0,1575.6,468.00,819.312,0,0,0,0
2,8/6/17,59388.0,1544.4,477.36,749.034,0,0,0,0
3,8/13/17,56964.0,1528.8,468.00,741.468,0,0,0,0
4,8/20/17,53460.0,1560.0,458.64,811.200,0,0,0,0


### 3. Feature Additions and Engineering - Change of Date

### 4. Exploratory Data Analysis and Statistical Analysis
We can have a detailed exploration that can be added to this section, but since we only need to answer three questions:

<b> 4.1 Exploring and Understanding basics data </b>

1. Distribution of Data across different accounts
2. Difference of Sales in Account Types (Using Categorical Mean)

<b> 4.2 Statistical Analysis - Answering the Questions</b>
1. Impact of Marketing Strategy on Sales (Using Correlation, Regression and Decision Tree)
2. Impact of Competition on Sales
3. How different types of client can have different strategies (Broken down Question 1 and Question 2 based on Account Type)

### 4.2 Impact of Marketing Strategy on Sales

#### Understanding of distrubtions

In [8]:
data.corr()[['sales']]

,sales
Week,0.102698
sales,1.000000
branded_search_spend,0.588992
nonbranded_search_spend,0.666117
facebook_spend,0.587216
print_spend,-0.228140
ooh_spend,-0.235758
tv_spend,0.393860
radio_spend,0.198385


Correlation Analysis

In [9]:
## Consolidated Strategy for Targeting

import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
correlation_analysis=pd.DataFrame(data[['sales',
'branded_search_spend', 'nonbranded_search_spend', 'facebook_spend',
       'print_spend', 'ooh_spend', 'tv_spend',
       'radio_spend']].corr()['sales']).reset_index()
correlation_analysis.columns=['Impacting Variable','Degree of Linear Impact (Correlation)']
correlation_analysis=correlation_analysis[correlation_analysis['Impacting Variable']!='sales']
correlation_analysis=correlation_analysis.sort_values('Degree of Linear Impact (Correlation)',ascending=False)
correlation_analysis.style.background_gradient(cmap=cm).set_precision(2)

<ipython-input-9-8eea419fc069>:12: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  correlation_analysis.style.background_gradient(cmap=cm).set_precision(2)


,Impacting Variable,Degree of Linear Impact (Correlation)
2,nonbranded_search_spend,0.67
1,branded_search_spend,0.59
3,facebook_spend,0.59
6,tv_spend,0.39
7,radio_spend,0.20
4,print_spend,-0.23
5,ooh_spend,-0.24


#### Regression Analysis (Market Sales and Strategies)

In [27]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
results = smf.ols('sales ~ nonbranded_search_spend + branded_search_spend + facebook_spend + facebook_spend + tv_spend + radio_spend + print_spend + ooh_spend',data=data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     82.37
Date:                Mon, 09 Oct 2023   Prob (F-statistic):           1.65e-61
Time:                        23:22:10   Log-Likelihood:                -2783.5
No. Observations:                 260   AIC:                             5583.
Df Residuals:                     252   BIC:                             5611.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                3

In [28]:
df = pd.read_html(results.summary().tables[1].as_html(),header=0,index_col=0)[0]

In [29]:
consolidated_summary=pd.DataFrame()

df=df.reset_index()
df=df[df['P>|t|']<0.05][['index','coef']]
df.columns=['Variable','Coefficent (Impact)']
df=df.sort_values('Coefficent (Impact)',ascending=False)
df=df[df['Variable']!='Intercept']

consolidated_summary=consolidated_summary.append(df)

df

<ipython-input-29-2fb0f03d054d>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  consolidated_summary=consolidated_summary.append(df)


,Variable,Coefficent (Impact)
1,nonbranded_search_spend,48.1103
4,tv_spend,5.7158
5,radio_spend,5.3235
6,print_spend,-7.6006


### 5. Final Recommendations

Using the below table we can use the coefficent to see how much return we can derive from each dollar we spend, here we can clearly see that for different account type different Campaigns and Different Sales Contact are effective with different extend.

<b>Case Explanation - Medium Facility </b><br>
For Example Medium Facility shows decent results with Flyer Campiagns and each dollar spend return 4 dollars on average. Sales Contact 2 is highly effective followed by Sales Contact 1 and Sales Contact 3. Else all other strategy shows no impact can be dropped to save cost.


In [30]:
consolidated_summary

,Variable,Coefficent (Impact)
1,nonbranded_search_spend,48.1103
4,tv_spend,5.7158
5,radio_spend,5.3235
6,print_spend,-7.6006


In [31]:
consolidated_summary.reset_index(inplace=True)
consolidated_summary.drop('index',inplace=True,axis=1)

In [32]:
consolidated_summary.columns = ['Variable','Return on Investment']
consolidated_summary['Return on Investment']= consolidated_summary['Return on Investment'].apply(lambda x: round(x,1))
consolidated_summary.style.background_gradient(cmap='RdYlGn')

,Variable,Return on Investment
0,nonbranded_search_spend,48.100000
1,tv_spend,5.700000
2,radio_spend,5.300000
3,print_spend,-7.600000


In [33]:
import seaborn as sns
import matplotlib.pyplot as plt

In [34]:
def format(x):
        return "${:.1f}".format(x)
consolidated_summary['Return on Investment']  = consolidated_summary['Return on Investment'].apply(format)

In [36]:
consolidated_summary.columns = ['Variable','Return on Investment']
consolidated_summary.style.background_gradient(cmap='RdYlGn')

,Variable,Return on Investment
0,nonbranded_search_spend,$48.1
1,tv_spend,$5.7
2,radio_spend,$5.3
3,print_spend,$-7.6
